This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [41]:
# First, import the relevant modules
import requests
from pandas.io.json import json_normalize
import pandas as pd

In [42]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2019-06-21&end_date=2019-06-21&api_key=ts4zFdKPe7kxg-4KJPkb'
r = requests.get(url)

# into the JSON structure that will be returned
json_data = (r.json())

In [43]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-06-25T22:37:05.976Z',
  'newest_available_date': '2019-06-25',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2019-06-21',
  'end_date': '2019-06-21',
  'data': [['2019-06-21',
    None,
    88.8,
    86.85,
    86.9,
    None,
    281537.0,
    24608655.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Question 2

In [44]:
#Call Quandi API for full year 
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key='
r = requests.get(url)

# into the JSON structure that will be returned
json_data = (r.json())


# Question 3

In [65]:
columns = json_data['dataset']['column_names']
columns

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [76]:
data = json_data['dataset']['data']

In [68]:
open_idx = columns.index('Open')
open_prices = []
for record in data:
    open_prices.append(record[open_idx])

In [70]:
open_prices = list(filter(None, open_prices))
max(open_prices), min(open_prices)

(53.11, 34.0)

# Question 4

In [74]:
high_idx = columns.index('High')
low_idx = columns.index('Low')
diff_prices = []
for record in data:
    diff_prices.append(record[high_idx]-record[low_idx])

In [75]:
diff_prices = list(filter(None, diff_prices))
max(diff_prices)

2.8100000000000023

# Question 5

In [77]:
close_idx = columns.index('Close')
close_prev = 0
close_diff = []
for record in data:
    differential = (record[close_idx]-close_prev)
    close_diff.append(differential)
    close_prev = record[close_idx]

In [78]:
close_diff = list(filter(None, close_diff))
max(close_diff)

51.76

# Question 7

In [80]:
trade_idx = columns.index('Traded Volume')
traded_volume = []
for record in data:
    traded_volume.append(record[trade_idx])

In [82]:
traded_volume = list(filter(None, traded_volume))

In [83]:
def Average(lst): 
    return sum(lst) / len(lst)

In [84]:
Average(traded_volume)

89124.33725490196

# Question 8

In [90]:
def getMedian(lst):
  values = sorted(lst)

  if len(values) % 2 == 1:
    return values[(len(values)+1)//2-1]
  else:
    lower = values[len(values)//2-1]
    upper = values[len(values)//2]
    return (float(lower + upper)) // 2 

In [92]:
getMedian(traded_volume)

76286.0